## Image processing for pheno-sudoku  
### Plan  

1. Ground truth:  
    a) Bin image to 4 chunks of 256x256;  
    b) Code for random selection of images from the screen (x20); perform LoG on channel 3, make RGB image, R/G/B = 80/10/10 (R = ch3, membrane); Code how to normalize channels for all the images;  
    c) create masks.  
2. Training of StarDist  
3. Segmentation 
    a) Import (use random selection)    
    b) Predict centroids and vertices  
    c) Estimate surface  
    d) Remove polygons under threshold  
    e) Export single cell img  
4. Clustering with unsupervised CNN  

When trained unsupervised net...  
1. visualize clusters
2. show normalized mutual information increase


**04/24/19 Update**  
All plates but 3:5,9,14 finished.  


### Deepcluster  
AlexNet arch was trained on nuclear crops from 1 field of view from every well of Pt04. For 100 epochs

## DL instance setup

```bash
conda create --name imgSudoku pandas numpy scipy seaborn opencv scikit-learn matplotlib tensorflow-gpu
conda activate imgSudoku

wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_10.1.105-1_amd64.deb

sudo dpkg -i cuda-repo-ubuntu1604_10.1.105-1_amd64.deb
sudo apt-key adv --fetch-keys http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/7fa2af80.pub
sudo apt-get update
sudo apt-get install cuda


```



### Agenda for 05/09/19  
**Synopsis:** we performed an image based screen on CRISPR-Cas9 induced knock outs of druggable genes. To analyze the phenotypic complexity of each knockout, I extracted single cell (nucleus) images using deep learning based segmentation. Total scale of data resulted in over 76 million single-nucleus images. To analyze phenotypic changes I used unsupervised deep learning and clustered the data (using deepcluster to train AlexNet model) in 100 arbitrary clusters. AlexNet was trained using 1 million single nucleus images.  

**Questions for a downstream analysis of the full dataset (alternatives):**  
1. Should I re-configure deepcluster-trained AlexNet to perform classification tasks on N pseudo labels? How to approach? I can remove the top layer and use input from last conv layer (4096 features) and map them to N clusters. Do you have recommendations how to re-arrange AlexNet in this case?  
2. I can extract 4096 features from conv5 for every image and perform the clustering. How should I go about this number of observations - loading full dataset and doing PCA seems infeasible? There are ways for incremental PCA clustering, not sure if they work with downstream UMAP. Also there is an online learning... What are the prerequisites to use either?  
3. How would you recommend to organize the infrastructure of this data? SPARK, Hadoop, SQL?  
4. Which stastistics to use for cluster evaluation?
5. Any recommendation of demonstrating interpretability of clustering net? Feature importance map?



### Update 05/07/19  
All plates finished and backed up (finished 05/03).  
How to process (alternative approaches)

1. K-means  
2. PCA/UMAP (with louvain and leiden)  
3. Unsupervised SVM  
4. Spectral clustering  

Benchmark all of them based on Pt04_r07_c05 (Incenp sgRNA1	190219_195942-V	G5)  

### Update 05/16/19  
Add also HDBSCAN  



#### K-means.  
Pre-trained AlexNet (described previously) run for 72 epochs to assign clusters for:  
    
    – Pt04_r07c05
    – Pt04_r07c05
    – Pt11_r02c12
    
Further training improved clustering with Louvain method (see `deepclust_multiple_samples.ipynb`).  


### Update 05/20/19  

#### Continuing training with PIC
Launched another training of 102-epochs pre-trained alex net on the following:  
```
Haus8 sgRNA #1	190219_195942-V	B12	
Haus8 sgRNA #2	190219_195942-V	E1	
Haus8 sgRNA #3	190219_195942-V	F2	
Haus8 sgRNA #4	190219_195942-V	G2	
DyncH1 sgRNA #3	190219_195942-V	A1	
Incenp sgRNA #1	190219_195942-V	G4	
Incenp sgRNA #2	190219_195942-V	G6	
Incenp sgRNA #3	190219_195942-V	H1	
Rad21 sgRNA #1	190219_195942-V	H3	
Rad21 sgRNA #2	190219_195942-V	H5	
Rad21 sgRNA #3	190219_195942-V	H7	
```
**NOTE** Testing PIC clustering for this one, since clusters are not balanced.

**Conclusion** After 15 epochs of re-training shows a fair cluster separation on evaluation data. Level of abstarction seems lower than k-means training. Results in https://github.com/Gvaihir/image_sudoku/blob/master/deepclust_test_PIC15ep.ipynb  
Started to train AlexNet with PIC further  


#### Continuing training with Kmeans (total 300 epochs)  
Launched another training of 102-epochs pre-trained alex net on the following:  
```
Haus8 sgRNA #1	190219_195942-V	B12	
Haus8 sgRNA #2	190219_195942-V	E1	
Haus8 sgRNA #3	190219_195942-V	F2	
Haus8 sgRNA #4	190219_195942-V	G2	
DyncH1 sgRNA #3	190219_195942-V	A1	
Incenp sgRNA #1	190219_195942-V	G4	
Incenp sgRNA #2	190219_195942-V	G6	
Incenp sgRNA #3	190219_195942-V	H1	
Rad21 sgRNA #1	190219_195942-V	H3	
Rad21 sgRNA #2	190219_195942-V	H5	
Rad21 sgRNA #3	190219_195942-V	H7	
```

### Additional strategy  
Extract features from multiple convolutional layers and cluster them

## References  
https://journals.aps.org/pre/pdf/10.1103/PhysRevE.80.056117 and https://www.nature.com/articles/srep30750 show Louvain to be most scalable  
https://arxiv.org/pdf/1708.02898.pdf set of benchmarks. Also idea of multiple layers for feature extraction

